In [10]:
import theano
import theano.tensor as T
import lasagne
import numpy as np
import os
import time
import utils
import cPickle as pickle
import BLSTM_sequence
import BLSTM_last
import DNN_liar
from theano.tensor.shared_randomstreams import RandomStreams

import sys
# from theano.printing import pydotprint




class Hierachi_RNN(object):
    def __init__(self, rnn_setting, val_split_ratio, D_batchsize, G_batchsize, liar_setting, 
                learning_rate1, learning_rate2, optimizer, 
                score_func_nonlin = 'default', wemb_trainable = 1, generator_halt_threshold = 0.1, wemb_size = None):
        # Initialize Theano Symbolic variable attributes
        self.story_input_variable = None
        self.story_mask = None
        self.story_nsent = 4

        self.cost = None
        self.learning_rate1 = float(learning_rate1)
        self.learning_rate2 = float(learning_rate2)
        self.classifier_train_func = None
        # Initialize data loading attributes
        self.wemb = theano.shared(np.random.rand(100, 300))
        self.val_set_path = '../../data/pickles/val_index_corpus.pkl'
        self.test_set_path = '../../data/pickles/test_index_corpus.pkl' 
        self.train_set_path = '../../data/pickles/train_index_corpus.pkl'
        self.wemb_matrix_path = '../../data/pickles/index_wemb_matrix.pkl'
        self.index2word_dict_path = '../../data/pickles/ROC_train_index_dict.pkl'

        self.rnn_units = int(rnn_setting)
        self.liar_setting = [int(elem) for elem in liar_setting.split('x')]
        # self.dropout_rate = float(dropout_rate)
        self.D_batchsize = int(D_batchsize)
        self.G_batchsize = int(G_batchsize)

        self.val_split_ratio = float(val_split_ratio)
        self.generator_halt_threshold = float(generator_halt_threshold)

        self.classifier_hid1 = 1024
        # self.val_split_ratio = float(val_split_ratio)
        self.words_num = 28820
        # self.delta = float(delta)
        if score_func_nonlin == 'default':
            self.score_func_nonlin = lasagne.nonlinearities.tanh
        else:
            self.score_func_nonlin = None

        self.wemb_trainable = bool(int(wemb_trainable))

        self.wemb_size = 300
        if wemb_size == None:
            self.random_init_wemb = False
        else:
            self.random_init_wemb = True
            self.wemb_size = int(wemb_size)

        self.train_story = None
        self.train_ending = None


        self.optimizer = optimizer

        self.val_story = None
        self.val_ending1 = None 
        self.val_ending2 = None
        self.val_answer = None
        self.n_val = None

        self.test_story = None 
        self.test_ending1 = None
        self.test_ending2 = None
        self.test_answer = None
        self.n_test = None

        self.sent_reps = []

        self.bilinear_attention_matrix = theano.shared(0.02*np.random.rand(self.rnn_units, self.rnn_units) - 0.01)


    def encoding_layer(self):


        assert len(self.reshaped_inputs_variables)==len(self.inputs_masks)
        for i in range(self.story_nsent):
            lstm_seq = lasagne.layers.get_output(self.encoder.output,
                                                        {self.encoder.l_in:self.reshaped_inputs_variables[i], 
                                                         self.encoder.l_mask:self.inputs_masks[i]},
                                                         deterministic = True)
            sent_rep = (lstm_seq * self.inputs_masks[i].dimshuffle(0,1,'x')).sum(axis = 1) / self.inputs_masks[i].sum(axis = 1, keepdims = True)
            self.sent_reps.append(sent_rep)

        self.end_seq = lasagne.layers.get_output(self.encoder.output,
                                            {self.encoder.l_in:self.reshaped_inputs_variables[4], 
                                            self.encoder.l_mask:self.inputs_masks[4]},
                                            deterministic = True)

        self.current_Nbatch = self.sent_reps[0].shape[0]

        merge_ls = [T.reshape(tensor, (tensor.shape[0], 1, tensor.shape[1])) for tensor in self.sent_reps]
        encode_merge = T.concatenate(merge_ls, axis = 1)

        self.plot_rep = lasagne.layers.get_output(self.sent_encoder.output, {self.sent_encoder.l_in: encode_merge}, 
                                                    deterministic = True)

    def attention_layer1(self):        
        n_batch, n_seq, _ = self.end_seq.shape

        #second attention
        bili_part1 = T.dot(self.end_seq, self.bilinear_attention_matrix)

        attention_score_tensor = T.batched_dot(bili_part1, self.plot_rep)

        numerator = self.inputs_masks[4] * T.exp(attention_score_tensor - attention_score_tensor.max(axis = 1, keepdims = True))
    
        attention_weight_matrix = numerator / numerator.sum(axis = 1, keepdims = True)

        attentioned_end_seq = self.end_seq * (attention_weight_matrix.reshape([n_batch, n_seq, 1]))

        self.attentioned_end_rep = T.sum(attentioned_end_seq, axis = 1) / T.sum(self.inputs_masks[4], axis = 1).reshape([-1, 1])

    def attention_layer2(self):
        n_batch, n_seq, _ = self.encode_2nd_end_seq.shape
        #second attention
        bili_part1 = T.dot(self.encode_2nd_end_seq, self.bilinear_attention_matrix)

        attention_score_tensor = T.batched_dot(bili_part1, self.plot_rep)

        numerator = self.vt_2nd_end_mask * T.exp(attention_score_tensor - attention_score_tensor.max(axis = 1, keepdims = True))
    
        attention_weight_matrix = numerator / numerator.sum(axis = 1, keepdims = True)

        attentioned_end_seq = self.encode_2nd_end_seq*(attention_weight_matrix.reshape([n_batch, n_seq, 1]))

        self.vt_2nd_end_repr = T.sum(attentioned_end_seq, axis = 1) / T.sum(self.vt_2nd_end_mask, axis = 1).reshape([-1, 1])

    def model_constructor(self, wemb_size = None):
        self.inputs_variables = []
        self.inputs_masks = []
        self.reshaped_inputs_variables = []
        self.vt_2nd_end_in = T.matrix('second_end', dtype='int64')
        self.vt_2nd_end = self.vt_2nd_end_in.reshape([self.vt_2nd_end_in.shape[0], self.vt_2nd_end_in.shape[1],1])
        self.vt_2nd_end_mask = T.matrix('second_end_mask', dtype=theano.config.floatX)

        for i in range(self.story_nsent+1):
            self.inputs_variables.append(T.matrix('story'+str(i)+'_input', dtype='int64'))
            self.inputs_masks.append(T.matrix('story'+str(i)+'_mask', dtype=theano.config.floatX))
            batch_size, seqlen = self.inputs_variables[i].shape
            self.reshaped_inputs_variables.append(self.inputs_variables[i].reshape([batch_size, seqlen, 1]))
            
        self.test1 = theano.function(self.inputs_variables + [self.vt_2nd_end_in], self.reshaped_inputs_variables + [self.vt_2nd_end])
        
        #initialize neural network units
        self.encoder = BLSTM_sequence.BlstmEncoder(LSTMLAYER_1_UNITS = self.rnn_units, wemb_trainable = self.wemb_trainable)
        self.encoder.build_model(self.wemb)
        self.sent_encoder = BLSTM_last.BlstmEncoder(INPUT_SIZE = self.rnn_units, LSTMLAYER_1_UNITS = self.rnn_units)
        self.sent_encoder.build_model()
        #build encoding layer
        self.encoding_layer()
        self.test2 = theano.function(self.inputs_variables + self.inputs_masks, [self.plot_rep, self.end_seq])

        '''============================================================================'''
        #encode possible second ending
        self.encode_2nd_end_seq = lasagne.layers.get_output(self.encoder.output,
                                                    {self.encoder.l_in:self.vt_2nd_end, 
                                                     self.encoder.l_mask:self.vt_2nd_end_mask},
                                                     deterministic = True)
        self.test3 = theano.function([self.vt_2nd_end_in, self.vt_2nd_end_mask], self.encode_2nd_end_seq)
        self.attention_layer1()
        self.test4 = theano.function(self.inputs_variables + self.inputs_masks, self.attentioned_end_rep)

        self.attention_layer2()
        self.test5 = theano.function(self.inputs_variables + [self.vt_2nd_end_in] + self.inputs_masks + [self.vt_2nd_end_mask], self.vt_2nd_end_repr, on_unused_input = 'ignore')
        '''============================================================================'''
        '''alternative ending construction part'''
        '''========================================================'''

        # self.DNN_liar.l_in.shape = (n_batch, self.rnn_units * 2)
        # self.DNN_liar.output = (n_batch, self.rnn_units)
        self.DNN_liar = DNN_liar.DNNLiar(INPUTS_SIZE = self.rnn_units, LAYER_UNITS = self.liar_setting, INPUTS_PARTS = 1)

        # self.alternative_end.shape = self.DNN_liar.output
        self.alternative_end = lasagne.layers.get_output(self.DNN_liar.output, {self.DNN_liar.l_in: self.plot_rep})

        '''========================================================'''

        '''discriminator'''
        '''========================================================'''

        l_story_in = lasagne.layers.InputLayer(shape=(None, self.rnn_units))
        l_end_in = lasagne.layers.InputLayer(shape = (None, self.rnn_units))
        l_concate = lasagne.layers.ConcatLayer([l_story_in, l_end_in], axis = 1)
        l_hid1 = lasagne.layers.DenseLayer(l_concate, num_units = self.classifier_hid1, nonlinearity=lasagne.nonlinearities.tanh)
        classifier = lasagne.layers.DenseLayer(l_hid1, num_units=2,
                                          nonlinearity=self.score_func_nonlin)

        final_class_param = lasagne.layers.get_all_params(classifier)

        '''========================================================'''
        '''               generator training graph                 '''
        '''========================================================'''
        # euclidien_distance = T.sqrt(T.sum(T.sqr(self.alternative_end - self.plot_rep), axis = 1))

        '''========================================================'''
        '''                    generating score                    '''
        '''========================================================'''

        # answer = (2 * srng.uniform((n_batch,))).astype('int64')

        # ending_pair_tensor1 = self.train_encodinglayer_vecs[-1] * (1-answer).dimshuffle(0,'x') + self.alternative_end * answer.dimshuffle(0,'x')
        # ending_pair_tensor2 = self.train_encodinglayer_vecs[-1] * (answer).dimshuffle(0,'x') + self.alternative_end * (1-answer).dimshuffle(0,'x')
        

        origi_score = lasagne.layers.get_output(classifier, {l_story_in: self.plot_rep, 
                                                   l_end_in: self.attentioned_end_rep})
        alter_score = lasagne.layers.get_output(classifier, {l_story_in: self.plot_rep, 
                                                   l_end_in: self.alternative_end})
        
        # srng = RandomStreams(seed=234)

        # noise_story = srng.normal((self.batchsize, self.rnn_units))
        # noise_score = lasagne.layers.get_output(classifier, {l_story_in: noise_story})

        '''========================================================'''

        vt_2nd_score = lasagne.layers.get_output(classifier, {l_story_in: self.plot_rep, 
                                                   l_end_in: self.vt_2nd_end_repr})

        self.test6 = theano.function(self.inputs_variables + [self.vt_2nd_end_in] + self.inputs_masks + [self.vt_2nd_end_mask], [origi_score, alter_score, vt_2nd_score], on_unused_input = 'ignore')

        prob1 = lasagne.nonlinearities.softmax(origi_score)
        prob2 = lasagne.nonlinearities.softmax(alter_score)

        # Construct symbolic cost function
        
        cost1 = lasagne.objectives.categorical_crossentropy(prob1, T.ones((self.current_Nbatch, )).astype('int64'))
        cost2 = lasagne.objectives.categorical_crossentropy(prob2, T.zeros((self.current_Nbatch, )).astype('int64'))

        liar_cost = lasagne.objectives.categorical_crossentropy(prob2, T.ones((self.current_Nbatch, )).astype('int64'))

        self.main_cost = lasagne.objectives.aggregate(cost1+cost2, mode = 'mean')
        self.liar_cost = lasagne.objectives.aggregate(liar_cost, mode = 'mean')

        # Retrieve all parameters from the network
        main_params = self.encoder.all_params + self.sent_encoder.all_params + final_class_param + [self.bilinear_attention_matrix]


        liar_params = self.DNN_liar.all_params

        main_updates = None
        liar_updates = None

        if self.optimizer == 'adam':
            main_updates = lasagne.updates.adam(self.main_cost, main_params, learning_rate=self.learning_rate1)

            liar_updates = lasagne.updates.adam(self.liar_cost, liar_params, learning_rate=self.learning_rate2)
        else:
            main_updates = lasagne.updates.momentum(self.main_cost, main_params, learning_rate=self.learning_rate1, momentum=0.9)
            liar_updates = lasagne.updates.momentum(self.liar_cost, liar_params, learning_rate=self.learning_rate2, momentum=0.9)
        # all_updates = lasagne.updates.momentum(self.cost, all_params, learning_rate = 0.05, momentum=0.9)

        # all_updates = []
        # for k,v in main_updates.items()+liar_updates.items():
        #     all_updates.append((k,v))

        # combine two sets of parameters update into a single OrderedDict 
        self.classifier_train_func = theano.function(self.inputs_variables + self.inputs_masks, 
                                        [self.main_cost, origi_score, alter_score], updates = main_updates)
        self.generator_train_func = theano.function(self.inputs_variables + self.inputs_masks, self.liar_cost, updates = liar_updates, on_unused_input='ignore')
        # Compute adam updates for training

        self.prediction = theano.function(self.inputs_variables + [self.vt_2nd_end_in] + self.inputs_masks + [self.vt_2nd_end_mask], [origi_score, vt_2nd_score])

        self.adv_monitor = theano.function(self.inputs_variables + self.inputs_masks, self.alternative_end, on_unused_input='ignore')

        self.test_end_matrix = T.matrix('test_end', dtype='int64')
        self.test_end_mask = T.matrix('test_end_mask', dtype=theano.config.floatX)


        self.test_end_rep = lasagne.layers.get_output(self.encoder.output,
                                                    {self.encoder.l_in:self.test_end_matrix, 
                                                    self.encoder.l_mask:self.test_end_mask},
                                                    deterministic = True)
        check_end_representation = (self.test_end_rep * self.test_end_mask.dimshuffle(0,1,'x')).sum(axis = 1) / self.test_end_mask.sum(axis = 1, keepdims = True)

        self.end_rep_check = theano.function([self.test_end_matrix, self.test_end_mask], check_end_representation)


In [11]:
a = Hierachi_RNN(300, 0.8, 150, 100, '512x256', 0.001, 0.001, 'sgd', 'default', 0, 0.01)
a.model_constructor()

In [12]:
import numpy as np
plot = [np.random.randint(5, size=(5,20)).astype('int64') for i in range(4)]
plot_mask = [np.ones((5,20)) for i in range(4)]
end1 = np.random.randint(5, size=(5,20)).astype('int64')
end1_mask = np.ones((5,20))
end2 = np.random.randint(5, size=(5,20)).astype('int64')
end2_mask = np.ones((5, 20))

results = a.test2(plot[0], plot[1], plot[2], plot[3], end1, plot_mask[0], plot_mask[1],  plot_mask[2],  plot_mask[3], end1_mask)
results[1].shape
#, plot_mask[0], plot_mask[1],  plot_mask[2],  plot_mask[3], end1_mask, end2_mask



(5, 20, 300)

In [13]:
result3 = a.test3(end2, end2_mask)
print result3.shape


(5, 20, 300)


In [14]:
result4 = a.test4(plot[0], plot[1], plot[2], plot[3], end1, plot_mask[0], plot_mask[1],  plot_mask[2],  plot_mask[3], end1_mask)

print result4.shape


(5, 300)


In [15]:
result5 = a.test5(plot[0], plot[1], plot[2], plot[3], end1, end2, plot_mask[0], plot_mask[1],  plot_mask[2],  plot_mask[3], end1_mask, end2_mask)

result5.shape

(5, 300)

In [18]:
result6 = a.prediction(plot[0], plot[1], plot[2], plot[3], end1, end2, plot_mask[0], plot_mask[1],  plot_mask[2],  plot_mask[3], end1_mask, end2_mask)
result6[0].shape

(5, 2)

In [8]:
import numpy as np
a = np.array([0.9,0.5,0.2])
b = np.array([0,1,1])
test(a,b)

array([ 2.30258509,  0.69314718,  1.60943791])

In [12]:
a = T.tensor3()
b = T.matrix()

c = T.mean(a, axis = 1)
d = T.sum(a, axis = 1) / T.sum(b, axis = 1).reshape([-1,1])

test1 = theano.function([a,b], [c,d])



In [14]:
a_1 = np.array([[[1,2,3], [4,5,6], [7,8,9], [10,11,12]],[[13,14,15],[16,17,18],[19,20,21],[22,23,24]]])
b_1 = np.array([[1,1,1], [2,2,2]])
mask_1 = np.array([[1,1,1,1],[1,1,1,0]])
test1(a_1, mask_1)

[array([[  5.5,   6.5,   7.5],
        [ 17.5,  18.5,  19.5]]),
 array([[  5.5       ,   6.5       ,   7.5       ],
        [ 23.33333333,  24.66666667,  26.        ]])]

In [402]:
import BLSTM_sequence
import sys
# from theano.printing import pydotprint




class Hierachi_RNN(object):
    def __init__(self, rnn_setting, dropout_rate, batchsize, val_split_ratio, wemb_size = None):
        # Initialize Theano Symbolic variable attributes
        self.story_input_variable = None
        self.story_mask = None
        self.story_nsent = 4

        self.cost = None

        self.train_func = None

        # Initialize data loading attributes
        self.wemb = theano.shared(np.random.rand(200, 300))
        self.val_set_path = '../../data/pickles/val_index_corpus.pkl'
        self.test_set_path = '../../data/pickles/test_index_corpus.pkl' 

        self.wemb_matrix_path = '../../data/pickles/index_wemb_matrix.pkl'

        self.rnn_units = int(rnn_setting)
        # self.mlp_units = [int(elem) for elem in mlp_setting.split('x')]
        self.bilinear_matrix = theano.shared(0.002*np.random.rand(self.rnn_units, self.rnn_units)-0.001)
        self.dropout_rate = float(dropout_rate)
        self.batchsize = int(batchsize)

        self.val_split_ratio = float(val_split_ratio)

        self.wemb_size = 300
        if wemb_size == None:
            self.random_init_wemb = False
        else:
            self.random_init_wemb = True
            self.wemb_size = int(wemb_size)

        self.train_story = None
        self.train_ending = None

        self.val_story = None
        self.val_ending1 = None 
        self.val_ending2 = None
        self.val_answer = None
        self.n_val = None

        self.test_story = None 
        self.test_ending1 = None
        self.test_ending2 = None
        self.test_answer = None
        self.n_test = None

        self.train_encodinglayer_vecs = []
        self.test_encodinglayer_vecs = []
        self.reasoninglayer_vec1 = []
        self.reasoninglayer_vec2 = []
        self.reasoninglayer_vec1_test = []
        self.reasoninglayer_vec2_test = []
        self.reasoning_pool_results = []
        self.reasoning_pool_results_test = []
        self.reasoners = []
        self.attentioned_sent_rep1 = []
        self.attentioned_sent_rep2 = []
        self.monitor1 = []
        self.monitor2 = []
        self.attention_moni1 = []
        self.attention_moni2 = []
        self.softmask_moni = []
        self.bilinear_attention_matrix = theano.shared(0.02*np.random.rand(self.rnn_units, self.rnn_units) - 0.01)

        
    def encoding_layer(self):


        assert len(self.reshaped_inputs_variables)==len(self.inputs_masks)
        for i in range(self.story_nsent):
            self.train_encodinglayer_vecs.append(lasagne.layers.get_output(self.encoder.output,
                                                        {self.encoder.l_in:self.reshaped_inputs_variables[i], 
                                                         self.encoder.l_mask:self.inputs_masks[i]},
                                                         deterministic = True))
        ending1_sequence_tensor = lasagne.layers.get_output(self.encoder.output,
                                                        {self.encoder.l_in:self.reshaped_inputs_variables[4],
                                                        self.encoder.l_mask:self.inputs_masks[4]},
                                                        deterministic = True)
        ending2_sequence_tensor = lasagne.layers.get_output(self.encoder.output,
                                                        {self.encoder.l_in:self.reshaped_inputs_variables[5],
                                                        self.encoder.l_mask:self.inputs_masks[5]},
                                                        deterministic = True)

        l_end_in= lasagne.layers.InputLayer((None, None, self.rnn_units))
        l_shuffle = lasagne.layers.DimshuffleLayer(l_end_in, (0,2,1))
        l_pooling = lasagne.layers.GlobalPoolLayer(l_shuffle)


        end1_representation = lasagne.layers.get_output(l_pooling, {l_end_in:ending1_sequence_tensor})
        end2_representation = lasagne.layers.get_output(l_pooling, {l_end_in:ending2_sequence_tensor})
        self.train_encodinglayer_vecs.append(end1_representation)
        self.train_encodinglayer_vecs.append(end2_representation)
        
    def attention_layer(self):        
        for i in range(self.story_nsent):
            n_batch, n_seq, _ = self.train_encodinglayer_vecs[i].shape

      
            bili_part1 = T.dot(self.train_encodinglayer_vecs[i], self.bilinear_attention_matrix)

            attention1_score_tensor = T.batched_dot(bili_part1, self.train_encodinglayer_vecs[4])

            attention2_score_tensor = T.batched_dot(bili_part1, self.train_encodinglayer_vecs[5])

#             softmax_mask = 
#             self.softmask_moni.append(softmax_mask)
            numerator1 = self.inputs_masks[i] * T.exp(attention1_score_tensor - attention1_score_tensor.max(axis = 1, keepdims = True))
            numerator2 = self.inputs_masks[i] * T.exp(attention2_score_tensor - attention2_score_tensor.max(axis = 1, keepdims = True))
        
            attention1_weight_matrix = numerator1 / numerator1.sum(axis = 1, keepdims = True)
            attention2_weight_matrix = numerator2 / numerator2.sum(axis = 1, keepdims = True)
#             attention1_weight_matrix = T.nnet.softmax(attention1_score_tensor.reshape([n_batch, -1])*self.inputs_masks[i])
#             attention2_weight_matrix = T.nnet.softmax(attention2_score_tensor.reshape([n_batch, -1])*self.inputs_masks[i])
            self.attention_moni1.append(attention1_weight_matrix)
            self.attention_moni2.append(attention2_weight_matrix)
            
            attentioned_sent_seq1 = self.train_encodinglayer_vecs[i]*(attention1_weight_matrix.reshape([n_batch, n_seq, 1]))
            attentioned_sent_seq2 = self.train_encodinglayer_vecs[i]*(attention2_weight_matrix.reshape([n_batch, n_seq, 1]))
            
            self.monitor1.append(attentioned_sent_seq1)
            self.monitor2.append(attentioned_sent_seq2)
            
            attentioned_sent_rep1 = T.sum(attentioned_sent_seq1, axis = 1) / T.sum(self.inputs_masks[i], axis = 1).reshape([-1, 1])
            attentioned_sent_rep2 = T.sum(attentioned_sent_seq2, axis = 1) / T.sum(self.inputs_masks[i], axis = 1).reshape([-1, 1])

            self.attentioned_sent_rep1.append(attentioned_sent_rep1)
            self.attentioned_sent_rep2.append(attentioned_sent_rep2)

           
    def model_constructor(self, wemb_size = None):
        self.inputs_variables = []
        self.inputs_masks = []
        self.reshaped_inputs_variables = []
        for i in range(self.story_nsent+2):
            self.inputs_variables.append(T.matrix('story'+str(i)+'_input', dtype='int64'))
            self.inputs_masks.append(T.matrix('story'+str(i)+'_mask', dtype=theano.config.floatX))
            batch_size, seqlen = self.inputs_variables[i].shape
            self.reshaped_inputs_variables.append(self.inputs_variables[i].reshape([batch_size, seqlen, 1]))

        #initialize neural network units
        self.encoder = BLSTM_sequence.BlstmEncoder(LSTMLAYER_1_UNITS = self.rnn_units, dropout_rate = self.dropout_rate)
        self.encoder.build_model(self.wemb)

        #build encoding layer
        self.encoding_layer()
        self.test0 = theano.function(self.inputs_variables + self.inputs_masks, self.train_encodinglayer_vecs)
        #build attention layer
        self.attention_layer()
        
        self.test2 = theano.function(self.inputs_variables + self.inputs_masks, self.monitor1+self.monitor2)
        self.test3 = theano.function(self.inputs_variables + self.inputs_masks, self.attention_moni1 + self.attention_moni2)
        
#         self.test4 = theano.function(self.inputs_masks[:4], self.softmask_moni)
        #build reasoning layers
        
        self.test1 = theano.function(self.inputs_variables + self.inputs_masks, self.attentioned_sent_rep1 + self.attentioned_sent_rep2)
        self.merge_ls1 = [T.reshape(tensor, (tensor.shape[0], 1, tensor.shape[1])) for tensor in self.attentioned_sent_rep1]
        self.merge_ls2 = [T.reshape(tensor, (tensor.shape[0], 1, tensor.shape[1])) for tensor in self.attentioned_sent_rep2]

        encode_merge1 = T.concatenate(self.merge_ls1, axis = 1)
        encode_merge2 = T.concatenate(self.merge_ls2, axis = 1)

        l_in = lasagne.layers.InputLayer(shape=(None, None, self.rnn_units))
        gate_parameters = lasagne.layers.recurrent.Gate(W_in=lasagne.init.Orthogonal(), 
                                                        W_hid=lasagne.init.Orthogonal(),
                                                        b=lasagne.init.Constant(0.))

        cell_parameters = lasagne.layers.recurrent.Gate(W_in=lasagne.init.Orthogonal(), 
                                                        W_hid=lasagne.init.Orthogonal(),
                                                        # Setting W_cell to None denotes that no cell connection will be used. 
                                                        W_cell=None, 
                                                        b=lasagne.init.Constant(0.),
                                                        # By convention, the cell nonlinearity is tanh in an LSTM. 
                                                        nonlinearity=lasagne.nonlinearities.tanh)

        l_lstm = lasagne.layers.recurrent.LSTMLayer(l_in, 
                                                    num_units=self.rnn_units,
                                                    # Here, we supply the gate parameters for each gate 
                                                    ingate=gate_parameters, forgetgate=gate_parameters, 
                                                    cell=cell_parameters, outgate=gate_parameters,
                                                    # We'll learn the initialization and use gradient clipping 
                                                    learn_init=True, grad_clipping=100.0)

        # The back directional LSTM layers
        l_lstm_back = lasagne.layers.recurrent.LSTMLayer(l_in,
                                                         num_units=self.rnn_units,
                                                         ingate=gate_parameters, forgetgate=gate_parameters, 
                                                         cell=cell_parameters, outgate=gate_parameters,
                                                         # We'll learn the initialization and use gradient clipping 
                                                         learn_init=True,grad_clipping=100.0,
                                                         backwards=True)

        # Do sum up of bidirectional LSTM results
        l_out_right = lasagne.layers.SliceLayer(l_lstm, -1, 1)
        l_out_left = lasagne.layers.SliceLayer(l_lstm_back, -1, 1)
        l_sum = lasagne.layers.ElemwiseSumLayer([l_out_right, l_out_left])

        reasoner_result1 = lasagne.layers.get_output(l_sum, {l_in: encode_merge1}, deterministic = True)
        reasoner_result2 = lasagne.layers.get_output(l_sum, {l_in: encode_merge2}, deterministic = True)

        reasoner_params = lasagne.layers.get_all_params(l_sum)

        # l_story_in = lasagne.layers.InputLayer(shape = (None, self.rnn_units))
        # l_end_in = lasagne.layers.InputLayer(shape = (None, self.rnn_units))
        # l_concate = lasagne.layers.ConcatLayer([l_story_in, l_end_in], axis = 1)

        # l_hid = lasagne.layers.DenseLayer(l_concate, num_units=2,
        #                                   nonlinearity=lasagne.nonlinearities.tanh)

        # final_class_param = lasagne.layers.get_all_params(l_hid)
        
        l_story_in = lasagne.layers.InputLayer(shape=(None, self.rnn_units))
        l_end_in = lasagne.layers.InputLayer(shape = (None, self.rnn_units))
        l_concate = lasagne.layers.ConcatLayer([l_story_in, l_end_in], axis = 1)

        l_hid = lasagne.layers.DenseLayer(l_concate, num_units=2,
                                          nonlinearity=lasagne.nonlinearities.tanh)

        final_class_param = lasagne.layers.get_all_params(l_hid)

        score1 = lasagne.layers.get_output(l_hid, {l_story_in: reasoner_result1, 
                                                   l_end_in: self.train_encodinglayer_vecs[-2]})
        score2 = lasagne.layers.get_output(l_hid, {l_story_in: reasoner_result2, 
                                                   l_end_in: self.train_encodinglayer_vecs[-1]})
        self.test5 = theano.function(self.inputs_variables + self.inputs_masks,[score1, score2])
        prob1 = lasagne.nonlinearities.softmax(score1)
        prob2 = lasagne.nonlinearities.softmax(score2)

        # Construct symbolic cost function
        target1 = T.vector('gold_target1', dtype= 'int64')
        target2 = T.vector('gold_target2', dtype= 'int64')
        
        cost1 = lasagne.objectives.categorical_crossentropy(prob1, target1)
        cost2 = lasagne.objectives.categorical_crossentropy(prob2, target2)

        self.cost = lasagne.objectives.aggregate(cost1+cost2, mode='sum')

        # Retrieve all parameters from the network
        all_params = self.encoder.all_params + reasoner_params + final_class_param

        all_updates = lasagne.updates.sgd(self.cost, all_params, learning_rate=0.001)
        # all_updates = lasagne.updates.momentum(self.cost, all_params, learning_rate = 0.05, momentum=0.9)
        updates_exp = [val for key,val in all_updates.items()]
        self.test_params = theano.function(self.inputs_variables+self.inputs_masks+[target1, target2], updates_exp, on_unused_input='warn')
        self.train_func = theano.function(self.inputs_variables + self.inputs_masks + [target1, target2], 
                                        [self.cost, prob1, prob2], updates = all_updates)

        # Compute adam updates for training

        self.prediction = theano.function(self.inputs_variables + self.inputs_masks, [score1, score2])
      

In [ ]:

a = Hierachi_RNN('300','0.0','20','0.25')
a.model_constructor()



In [400]:
v = [[],[],[],[],[],[]]
for i in range(6):
    for j in range(5):
        v[i].append(np.random.randint(100, size=(leng_v[i+j], )))
        
inputs_v = [utils.padding(sentence) for sentence in v]

inputs_m = [utils.mask_generator(sentence) for sentence in v]

targ1 = np.random.randint(2, size=(5,))
targ2 = 1 - targ1
params = a.train_func(inputs_v[0], inputs_v[1], inputs_v[2], inputs_v[3], inputs_v[4], inputs_v[5],
                                  inputs_m[0], inputs_m[1], inputs_m[2], inputs_m[3], inputs_m[4], inputs_m[5], targ1, targ2)



In [401]:
params[0]

array(7.641996219323214)

In [328]:
score1= a.test2(inputs_v[0], inputs_v[1], inputs_v[2], inputs_v[3], inputs_v[4], inputs_v[5],
                                  inputs_m[0], inputs_m[1], inputs_m[2], inputs_m[3], inputs_m[4], inputs_m[5])





In [332]:
inputs_v

[array([[28, 54, 37, 81, 45, 85, 43, 30,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 45,  7, 27, 75, 20, 19, 50,  0, 61, 89, 68, 88, 63, 27,  5],
        [49, 47, 95, 73, 22, 82, 58, 60, 94, 66,  6, 38, 87, 38, 67,  0],
        [57, 51,  4, 97, 44, 47, 61, 35, 10, 62, 92,  0,  0,  0,  0,  0],
        [ 1, 61, 44, 99, 33, 92, 71, 81, 57, 10, 59, 26, 39, 91, 66, 29]]),
 array([[93, 81, 48, 80, 88, 74, 81, 96, 98, 67, 59, 72, 51, 93, 17, 78],
        [66, 13, 22, 10, 19, 47, 10, 81, 57, 43, 78, 56, 90, 11, 84,  0],
        [ 4, 33, 74, 10, 97,  2,  0, 86, 93, 51, 52,  0,  0,  0,  0,  0],
        [19, 22, 74, 43, 85, 97, 49, 46, 80, 95, 72, 18, 13, 13, 48, 48],
        [54, 35, 99, 72, 19, 95, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0]]),
 array([[44, 22, 18, 66, 27, 56,  9, 41,  0, 26, 78, 41, 15, 85,  6,  0],
        [ 8, 79, 77, 60,  3, 53, 39, 11, 87, 26, 71,  0,  0,  0,  0,  0],
        [ 8, 87, 11, 57, 76, 87, 51, 82, 90, 10, 56, 16, 14, 14, 13, 23],
        [10, 22, 44, 56, 18, 77, 7

In [178]:
import utils
leng_v = np.random.randint(5, 17, size=(30))

v = [[],[],[],[],[],[]]
for i in range(6):
    for j in range(5):
        v[i].append(np.random.randint(100, size=(leng_v[i+j], )))
        
inputs_v = [utils.padding(sentence) for sentence in v]

inputs_m = [utils.mask_generator(sentence) for sentence in v]

result = a.test2(inputs_v[0], inputs_v[1], inputs_v[2], inputs_v[3], inputs_v[4], inputs_v[5],
                 inputs_m[0], inputs_m[1], inputs_m[2], inputs_m[3], inputs_m[4], inputs_m[5])

In [200]:
print inputs_v[0]
print inputs_m[0]
print result[0][3][10]

[[11  4 55 67 81 81  6 29  0  0  0  0  0  0  0  0]
 [21 36 34 19 78 49  2 19  0 77 75 74 17  9 19  3]
 [ 5 29 11 94 21 70 78 10  2  6 98 42 66 28  8  0]
 [ 3 67 35 72 27 50 47 96 91 89 98  0  0  0  0  0]
 [44 30 99 88 56 79 69 91 34 33 72 41 28 81 74 33]]
[[ 1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]
[  6.12901829e-04   1.28206219e-02   1.85810721e-02   1.77323277e-03
  -1.10088005e-02  -5.78050119e-04   9.91322895e-04   1.83338379e-02
  -7.76145479e-03   9.97180350e-03   2.69632479e-03  -1.11936929e-02
  -6.97481570e-03   6.67866176e-03  -6.61085645e-04   8.17177092e-03
  -8.19279369e-03   1.05830154e-02  -1.43372396e-02  -2.16162561e-03
   3.08066795e-03   5.19928999e-03   5.88627218e-04  -7.8830421

In [150]:

IN_ = T.matrix(dtype = 'int64')
MASK = T.matrix(dtype = theano.config.floatX)

IN = IN_.reshape((IN_.shape[0], IN_.shape[1], 1))

wemb = theano.shared(np.random.rand(100, 300))
l_in = lasagne.layers.InputLayer(shape=(None, None, 1))

# Masks input shape ==> (n_batch, n_time_steps)
l_mask = lasagne.layers.InputLayer(shape=(None, None))

#setting gates and cell parameters with specific nonlinearity functions
gate_parameters = lasagne.layers.recurrent.Gate(W_in=lasagne.init.Orthogonal(), 
                                                W_hid=lasagne.init.Orthogonal(),
                                                b=lasagne.init.Constant(0.))

cell_parameters = lasagne.layers.recurrent.Gate(W_in=lasagne.init.Orthogonal(), 
                                                W_hid=lasagne.init.Orthogonal(),
                                                # Setting W_cell to None denotes that no cell connection will be used. 
                                                W_cell=None, 
                                                b=lasagne.init.Constant(0.),
                                                # By convention, the cell nonlinearity is tanh in an LSTM. 
                                                nonlinearity=lasagne.nonlinearities.tanh)

# The embedding layers with retieve subtensor from word embedding matrix
l_emb = lasagne.layers.EmbeddingLayer(l_in, input_size=wemb.get_value().shape[0], output_size=wemb.get_value().shape[1], W=wemb)

l_drop = lasagne.layers.DropoutLayer(l_emb, p = 0.0)
# The LSTM layer should have the same mask input in order to avoid padding entries
l_lstm = lasagne.layers.recurrent.LSTMLayer(l_drop, 
                                            num_units=300,
                                            # We need to specify a separate input for masks
                                            mask_input=l_mask,
                                            # Here, we supply the gate parameters for each gate 
                                            ingate=gate_parameters, forgetgate=gate_parameters, 
                                            cell=cell_parameters, outgate=gate_parameters,
                                            # We'll learn the initialization and use gradient clipping 
                                            learn_init=True
                                            )


# The back directional LSTM layers
l_lstm_back = lasagne.layers.recurrent.LSTMLayer(l_drop,
                                                 num_units=300,
                                                 mask_input = l_mask,
                                                 ingate=gate_parameters, forgetgate=gate_parameters, 
                                                 cell=cell_parameters, outgate=gate_parameters,
                                                 # We'll learn the initialization and use gradient clipping 
                                                 learn_init=True,
                                                 backwards=True
                                                )


# Do sum up of bidirectional LSTM results
l_sum = lasagne.layers.ElemwiseSumLayer([l_lstm, l_lstm_back])
out = lasagne.layers.get_output(l_sum, {l_in:IN, l_mask:MASK}, deterministic = True)

test = theano.function([IN_, MASK], out)

In [174]:
result = test(inputs_v[5], inputs_m[5])
print result[4][9]

[  1.41805189e-01   4.65947280e-02  -8.36854174e-02   1.07350674e-01
  -2.37396935e-01  -1.79823246e-01   6.51879467e-02   4.60637026e-03
  -2.09297061e-01  -1.59060352e-01   2.93781539e-01   2.85348498e-01
   8.93948845e-02   5.16647735e-02  -6.63083297e-01  -3.32605698e-01
  -2.13643046e-01  -2.52780222e-02   3.30183841e-01  -1.86089749e-01
   2.64025800e-01   1.07492811e-01   5.57596003e-02  -4.45173354e-01
  -1.89919893e-01   1.83129174e-01   1.55992846e-01   6.95604518e-02
   2.86514647e-01  -4.01425141e-01  -3.06143785e-02  -2.88565510e-01
  -6.13725809e-01  -1.95180382e-01  -2.38842957e-01  -2.73882412e-01
  -1.94990750e-02   1.82266075e-01  -2.67372722e-02   1.89460599e-01
  -3.17284749e-01   1.24459833e-01   1.97643377e-02  -4.07787518e-02
   1.33853864e-01  -1.82464355e-01   8.99734217e-02   3.00601495e-01
   1.07841359e-01   1.00602480e-01  -1.10121118e-01  -1.49100513e-01
   4.55662451e-02   3.19698477e-01  -2.52166452e-01  -5.48263467e-02
  -1.66052223e-01   2.32730964e-01

In [30]:
a = T.matrix()

b = T.sum(a, axis = 1).reshape([-1,1])

c = T.matrix()
d = c / b
test = theano.function([a,c], d)

In [32]:
test(np.array([[1,1]]), np.array([[3,3]]))

array([[ 1.5,  1.5]])

In [224]:
a = T.tensor3()

b = theano.shared
a_ = a.reshape((-1, a.shape[2]))
b_ = b.reshape((1,b.shape[0],b.shape[1]))
b_f = b_ + T.zeros((a_.shape[0], b.shape[0], b.shape[1]))
result = T.batched_dot(a_, b_f).reshape((a.shape[0], a.shape[1], -1))
loss = result.sum()

test = theano.function([a,b], result)
train  = theano.function([a,b], )

In [225]:
a_test = np.arange(3*4*5).reshape((3,4,5))

b_test = np.arange(5*5).reshape((5,5))

testing = test(a_test, b_test)
print testing

ValueError: Shape mismatch: batch sizes unequal. x.shape is (12, 1, 5), y.shape is (1, 5, 5).
Apply node that caused the error: BatchedDot(Reshape{2}.0, Reshape{3}.0)
Toposort index: 10
Inputs types: [TensorType(float64, matrix), TensorType(float64, (True, False, False))]
Inputs shapes: [(12, 5), (1, 5, 5)]
Inputs strides: [(40, 8), (200, 40, 8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Reshape{3}(BatchedDot.0, MakeVector{dtype='int64'}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 276, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 228, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 391, in execute_request
    user_expressions, allow_stdin)
  File "/usr/local/lib/python2.7

In [7]:
import numpy as np
import theano
import theano.tensor as T

c = np.ones((5,5))
c[0] = 0.0
a = theano.shared(c)
b = T.matrix()
d = b*a
test = theano.function([b], d)

In [17]:
a = T.matrix()
b = T.roll(a, shift = -4, axis = 0)
test = theano.function([a],b)

In [18]:
a_t = np.arange(20).reshape(4,5)
test(a_t)

array([[  0.,   1.,   2.,   3.,   4.],
       [  5.,   6.,   7.,   8.,   9.],
       [ 10.,  11.,  12.,  13.,  14.],
       [ 15.,  16.,  17.,  18.,  19.]])

In [245]:
a = np.array([1,2,3,4])
b = np.array([1,2,3,3])
c = np.all(a-b==0)
print c

False


In [95]:
a = np.arange(3 * 4 * 5).reshape(3, 4, 5).astype('float32')
b = np.arange(3 * 5).reshape(3, 5).astype('float32')

A = T.ftensor3()
B = T.fmatrix()

# out1 = T.tensordot(A,B)
out = (A * B.dimshuffle(0, 'x', 1)).sum(2)
print a
print b
out.eval({A: a, B: b})
# out1.eval({A:a, B:b})


[[[  0.   1.   2.   3.   4.]
  [  5.   6.   7.   8.   9.]
  [ 10.  11.  12.  13.  14.]
  [ 15.  16.  17.  18.  19.]]

 [[ 20.  21.  22.  23.  24.]
  [ 25.  26.  27.  28.  29.]
  [ 30.  31.  32.  33.  34.]
  [ 35.  36.  37.  38.  39.]]

 [[ 40.  41.  42.  43.  44.]
  [ 45.  46.  47.  48.  49.]
  [ 50.  51.  52.  53.  54.]
  [ 55.  56.  57.  58.  59.]]]
[[  0.   1.   2.   3.   4.]
 [  5.   6.   7.   8.   9.]
 [ 10.  11.  12.  13.  14.]]


array([[   30.,    80.,   130.,   180.],
       [  780.,   955.,  1130.,  1305.],
       [ 2530.,  2830.,  3130.,  3430.]], dtype=float32)

In [263]:
a = T.matrix()
a_ = T.nnet.softmax(a)
b = T.imatrix()

c = lasagne.objectives.categorical_crossentropy(a_, b)
test = theano.function([a, b], c)

In [274]:
import cPickle as pickle
dictionary = pickle.load(open('../../data/pickles/index_wemb_matrix.pkl'))

In [275]:
dictionary[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [390]:
x = T.matrix()

e_x = x.max(axis=1, keepdims=True)

test = theano.function([x], e_x)

In [3]:
import theano
import theano.tensor as T

import numpy as np
wemb = theano.shared(np.arange(100*5).reshape(100,5))
a = T.matrix()

b = T.iscalar()

c = T.arange(b)

d = T.set_subtensor(a[c], wemb[c])

test = theano.function([a,b], [a,d])




In [4]:
a_test = np.ones((7,5))
b = 3
test(a_test, b)

[array([[ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.]]), array([[  0.,   1.,   2.,   3.,   4.],
        [  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.],
        [  1.,   1.,   1.,   1.,   1.],
        [  1.,   1.,   1.,   1.,   1.],
        [  1.,   1.,   1.,   1.,   1.],
        [  1.,   1.,   1.,   1.,   1.]])]

In [7]:
srng = T.shared_randomstreams.RandomStreams(seed=234)
aa = T.iscalar()
substi_word_index = (2 * srng.uniform((aa, ))).astype('int64')
test = theano.function([aa], substi_word_index)

In [8]:
test(1)

array([0])

In [83]:
import BLSTM_sequence
import BLSTM_last
import lasagne
import theano
import theano.tensor as T
import numpy as np

class Hierachi_RNN(object):
    def __init__(self, rnn_setting, batchsize, wemb_size = None):
        # Initialize Theano Symbolic variable attributes
        self.story_input_variable = None
        self.story_mask = None
        self.story_nsent = 4

        self.cost = None

        self.train_func = None

        # Initialize data loading attributes
        self.wemb = theano.shared(np.random.rand(300, 300))
        self.val_set_path = '../../data/pickles/val_index_corpus.pkl'
        self.test_set_path = '../../data/pickles/test_index_corpus.pkl' 
        self.train_set_path = '../../data/pickles/train_index_corpus.pkl'
        self.wemb_matrix_path = '../../data/pickles/index_wemb_matrix.pkl'

        self.rnn_units = int(rnn_setting)
        # self.mlp_units = [int(elem) for elem in mlp_setting.split('x')]
        self.bilinear_matrix = theano.shared(0.002*np.random.rand(self.rnn_units, self.rnn_units)-0.001)
        # self.dropout_rate = float(dropout_rate)
        self.batchsize = int(batchsize)

        # self.val_split_ratio = float(val_split_ratio)
        self.words_num = 100

        self.wemb_size = 300
        if wemb_size == None:
            self.random_init_wemb = False
        else:
            self.random_init_wemb = True
            self.wemb_size = int(wemb_size)

        self.train_story = None
        self.train_ending = None

        self.val_story = None
        self.val_ending1 = None 
        self.val_ending2 = None
        self.val_answer = None
        self.n_val = None

        self.test_story = None 
        self.test_ending1 = None
        self.test_ending2 = None
        self.test_answer = None
        self.n_test = None

        self.train_encodinglayer_vecs = []
        self.test_encodinglayer_vecs = []
        self.reasoninglayer_vec1 = []
        self.reasoninglayer_vec2 = []
        self.reasoninglayer_vec1_test = []
        self.reasoninglayer_vec2_test = []
        self.reasoning_pool_results = []
        self.reasoning_pool_results_test = []
        self.reasoners = []
        self.attentioned_sent_rep1 = []
        self.attentioned_sent_rep2 = []
        self.bilinear_attention_matrix = theano.shared(0.02*np.random.rand(self.rnn_units, self.rnn_units) - 0.01)

    def encoding_layer(self):


        assert len(self.reshaped_inputs_variables)==len(self.inputs_masks)
        for i in range(self.story_nsent + 1):
            self.train_encodinglayer_vecs.append(lasagne.layers.get_output(self.encoder.output,
                                                        {self.encoder.l_in:self.reshaped_inputs_variables[i], 
                                                         self.encoder.l_mask:self.inputs_masks[i]},
                                                         deterministic = True))


        end_representation = (self.train_encodinglayer_vecs[-1] * self.inputs_masks[4].dimshuffle(0,1,'x')).sum(axis = 1) / self.inputs_masks[4].sum(axis = 1, keepdims = True)

        #The last one (index-5) is the average of the non-attentioned ending sequence
        self.train_encodinglayer_vecs.append(end_representation)
        
    def attention1_layer(self):        
        for i in range(self.story_nsent):
            n_batch, n_seq, _ = self.train_encodinglayer_vecs[i].shape

            #second attention

            bili_part1 = T.dot(self.train_encodinglayer_vecs[i], self.bilinear_attention_matrix)

            attention1_score_tensor = T.batched_dot(bili_part1, self.train_encodinglayer_vecs[5])

            numerator1 = self.inputs_masks[i] * T.exp(attention1_score_tensor - attention1_score_tensor.max(axis = 1, keepdims = True))
        
            attention1_weight_matrix = numerator1 / numerator1.sum(axis = 1, keepdims = True)

            attentioned_sent_seq1 = self.train_encodinglayer_vecs[i]*(attention1_weight_matrix.reshape([n_batch, n_seq, 1]))

            attentioned_sent_rep1 = T.sum(attentioned_sent_seq1, axis = 1) / T.sum(self.inputs_masks[i], axis = 1).reshape([-1, 1])

            self.attentioned_sent_rep1.append(attentioned_sent_rep1)

    def attention2_layer(self):
        for i in range(self.story_nsent):
            n_batch, n_seq, _ = self.train_encodinglayer_vecs[i].shape

            #second attention

            bili_part1 = T.dot(self.train_encodinglayer_vecs[i], self.bilinear_attention_matrix)

            attention2_score_tensor = T.batched_dot(bili_part1, self.vt_2nd_end_repr)

            numerator2 = self.inputs_masks[i] * T.exp(attention2_score_tensor - attention2_score_tensor.max(axis = 1, keepdims = True))
        
            attention2_weight_matrix = numerator2 / numerator2.sum(axis = 1, keepdims = True)

            attentioned_sent_seq2 = self.train_encodinglayer_vecs[i]*(attention2_weight_matrix.reshape([n_batch, n_seq, 1]))

            attentioned_sent_rep2 = T.sum(attentioned_sent_seq2, axis = 1) / T.sum(self.inputs_masks[i], axis = 1).reshape([-1, 1])

            self.attentioned_sent_rep2.append(attentioned_sent_rep2)
           
    def model_constructor(self, wemb_size = None):
        self.inputs_variables = []
        self.inputs_masks = []
        self.reshaped_inputs_variables = []
        self.vt_2nd_end = T.matrix('second_end', dtype='int64')
        self.vt_2nd_end_mask = T.matrix('second_end_mask', dtype=theano.config.floatX)

        for i in range(self.story_nsent+1):
            self.inputs_variables.append(T.matrix('story'+str(i)+'_input', dtype='int64'))
            self.inputs_masks.append(T.matrix('story'+str(i)+'_mask', dtype=theano.config.floatX))
            batch_size, seqlen = self.inputs_variables[i].shape
            self.reshaped_inputs_variables.append(self.inputs_variables[i].reshape([batch_size, seqlen, 1]))

        #initialize neural network units
        self.encoder = BLSTM_sequence.BlstmEncoder(LSTMLAYER_1_UNITS = self.rnn_units)
        self.encoder.build_model(self.wemb)

        #build encoding layer
        self.encoding_layer()

        '''============================================================================'''
        #encode possible second ending
        self.encode_2nd_end_seq = lasagne.layers.get_output(self.encoder.output,
                                                    {self.encoder.l_in:self.vt_2nd_end, 
                                                     self.encoder.l_mask:self.vt_2nd_end_mask},
                                                     deterministic = True)

        self.vt_2nd_end_repr = (self.encode_2nd_end_seq * self.vt_2nd_end_mask.dimshuffle(0,1,'x')).sum(axis = 1) / self.vt_2nd_end_mask.sum(axis = 1, keepdims = True)
        '''============================================================================'''

        #build attention layer
        self.attention1_layer()
        self.attention2_layer()
        #build reasoning layers
        
        self.test1 = theano.function(self.inputs_variables + self.inputs_masks, self.train_encodinglayer_vecs[5], on_unused_input='ignore')
        # merge tensors to fit in BLSTM models as input tensor
        # merge_ls1.shape = (n_batch, m_seq, self.rnn_units)
        self.merge_ls1 = [T.reshape(tensor, (tensor.shape[0], 1, tensor.shape[1])) for tensor in self.attentioned_sent_rep1]
        self.merge_ls2 = [T.reshape(tensor, (tensor.shape[0], 1, tensor.shape[1])) for tensor in self.attentioned_sent_rep2]
        encode_merge1 = T.concatenate(self.merge_ls1, axis = 1)
        encode_merge2 = T.concatenate(self.merge_ls2, axis = 1)
        self.test2 = theano.function(self.inputs_variables + [self.vt_2nd_end] + self.inputs_masks + [self.vt_2nd_end_mask], [encode_merge1, encode_merge2])
        # create sentence level encoder
        # using only last encoding result of both direction of the LSTM
        self.sent_encoder = BLSTM_last.BlstmEncoder(INPUT_SIZE = self.rnn_units, LSTMLAYER_1_UNITS = self.rnn_units)
        self.sent_encoder.build_model()

        # reasoner_result1.shape = (n_batch, self.rnn_units)
        reasoner_result1 = lasagne.layers.get_output(self.sent_encoder.output, {self.sent_encoder.l_in: encode_merge1}, 
                                                    deterministic = True)

        reasoner_result2 = lasagne.layers.get_output(self.sent_encoder.output, {self.sent_encoder.l_in: encode_merge2}, 
                                                    deterministic = True)
        self.test3 = theano.function(self.inputs_variables + [self.vt_2nd_end] + self.inputs_masks + [self.vt_2nd_end_mask], [reasoner_result1, reasoner_result2])
        # self.train_encodinglayer_vecs[4].shape == (n_batch, m_seq, 300)
        end_attention_part1 = T.dot(self.train_encodinglayer_vecs[4], self.bilinear_attention_matrix)
        end_attention = T.batched_dot(end_attention_part1, reasoner_result1)


        end_att_numerator1 = self.inputs_masks[4] * T.exp(end_attention - end_attention.max(axis = 1, keepdims = True))
    
        end_attention_weight_matrix = end_att_numerator1 / end_att_numerator1.sum(axis = 1, keepdims = True)
        self.test3_1 = theano.function(self.inputs_variables + self.inputs_masks, end_attention_weight_matrix)
        # end_attention_weight_matrix.shape = (n_batch, m_seq)
        # neg_ending_rep.shape == (n_batc, m_seq)
        # neg_ending_subs_index is the one to be substituted in the original sentence
        neg_ending_subs_index = T.argmax(end_attention_weight_matrix, axis = 1)
        n_batch = self.train_encodinglayer_vecs[0].shape[0]

        # substi_word_index is the random index generate to surrogate the original ones
        srng = T.shared_randomstreams.RandomStreams(seed=234)
        substi_word_index = (self.words_num * srng.uniform((n_batch, ))).astype('int64')
        self.test3_2 = theano.function(self.inputs_variables + self.inputs_masks, substi_word_index, on_unused_input = 'ignore')
        batch_dim = T.arange(n_batch)
        self.test3_3 = theano.function(self.inputs_variables + self.inputs_masks, batch_dim, on_unused_input = 'ignore')
        # set_subtensor won't change the original tensor
        # new_end_seq_tensor.shape = (self.batchsize, m_seq, 300)
        new_end_seq_tensor = T.set_subtensor(self.train_encodinglayer_vecs[4][batch_dim, neg_ending_subs_index], self.wemb[substi_word_index])
        # neg_end_rep.shape = (self.batchsize, self.rnn_units)
        self.test3_4 = theano.function(self.inputs_variables + self.inputs_masks, new_end_seq_tensor)
        neg_end_rep = (new_end_seq_tensor * self.inputs_masks[4].dimshuffle(0,1,'x')).sum(axis = 1) / self.inputs_masks[4].sum(axis = 1, keepdims = True)
        self.test4 = theano.function(self.inputs_variables + self.inputs_masks, neg_end_rep)
        
        l_story_in = lasagne.layers.InputLayer(shape=(None, self.rnn_units))
        l_end_in = lasagne.layers.InputLayer(shape = (None, self.rnn_units))
        l_concate = lasagne.layers.ConcatLayer([l_story_in, l_end_in], axis = 1)
        
        l_hid = lasagne.layers.DenseLayer(l_concate, num_units=2,
                                          nonlinearity=lasagne.nonlinearities.tanh)

        final_class_param = lasagne.layers.get_all_params(l_hid)
        answer = (2 * srng.uniform((n_batch,))).astype('int64')

        ending_pair_tensor1 = self.train_encodinglayer_vecs[-1] * (1-answer).dimshuffle(0,'x') + neg_end_rep * answer.dimshuffle(0,'x')
        ending_pair_tensor2 = self.train_encodinglayer_vecs[-1] * (answer).dimshuffle(0,'x') + neg_end_rep * (1-answer).dimshuffle(0,'x')
        self.test5 = theano.function(self.inputs_variables + self.inputs_masks, [ending_pair_tensor1, ending_pair_tensor2])

        score1 = lasagne.layers.get_output(l_hid, {l_story_in: reasoner_result1, 
                                                   l_end_in: ending_pair_tensor1})
        score2 = lasagne.layers.get_output(l_hid, {l_story_in: reasoner_result1, 
                                                   l_end_in: ending_pair_tensor2})

        vt_2nd_score = lasagne.layers.get_output(l_hid, {l_story_in: reasoner_result2, 
                                                   l_end_in: self.vt_2nd_end_repr})
        self.test6 = theano.function(self.inputs_variables + [self.vt_2nd_end] + self.inputs_masks + [self.vt_2nd_end_mask], [score1, score2, vt_2nd_score])


        prob1 = lasagne.nonlinearities.softmax(score1)
        prob2 = lasagne.nonlinearities.softmax(score2)


        # Construct symbolic cost function
        
        cost1 = lasagne.objectives.categorical_crossentropy(prob1, answer)
        cost2 = lasagne.objectives.categorical_crossentropy(prob2, 1-answer)

        self.cost = lasagne.objectives.aggregate(cost1+cost2, mode='sum')

        # Retrieve all parameters from the network
        all_params = self.encoder.all_params + self.sent_encoder.all_params + final_class_param + [self.bilinear_attention_matrix]

        all_updates = lasagne.updates.adam(self.cost, all_params, learning_rate=0.001)
        # all_updates = lasagne.updates.momentum(self.cost, all_params, learning_rate = 0.05, momentum=0.9)

        self.train_func = theano.function(self.inputs_variables + self.inputs_masks, 
                                        [self.cost, prob1, prob2, answer], updates = all_updates)

        # Compute adam updates for training

        self.prediction = theano.function(self.inputs_variables + [self.vt_2nd_end] + self.inputs_masks + [self.vt_2nd_end_mask], [score1, vt_2nd_score])
        # pydotprint(self.train_func, './computational_graph.png')


In [84]:
a = Hierachi_RNN(rnn_setting = '300', batchsize = '20')

a.model_constructor()

In [85]:
import utils
leng_v = np.random.randint(5, 17, size=(30))

v = [[],[],[],[],[],[]]
for i in range(6):
    for j in range(5):
        v[i].append(np.random.randint(100, size=(leng_v[i+j], )))
        
inputs_v = [utils.padding(sentence) for sentence in v]

inputs_m = [utils.mask_generator(sentence) for sentence in v]

result = a.test3_4(inputs_v[0], inputs_v[1], inputs_v[2], inputs_v[3], inputs_v[4],
                 inputs_m[0], inputs_m[1], inputs_m[2], inputs_m[3], inputs_m[4])



In [88]:
#one line cases
leng_v = np.random.randint(5, 17, size=(30))
test_inputs = [np.random.randint(100, size=(1,leng_v[j])) for i in range(6) for j in range(6)]

test_masks = [np.ones_like(sent) for sent in test_inputs]

result = a.test3_4(test_inputs[0],test_inputs[1],test_inputs[2],test_inputs[3],test_inputs[4],
                test_masks[0], test_masks[1], test_masks[2], test_masks[3],test_masks[4])


In [90]:
result[0].shape

(9, 300)

In [46]:
a = T.tensor3()
n_batch = a.shape[0]
neg_ending_subs_index = T.ivector()
srng = T.shared_randomstreams.RandomStreams(seed=234)
substi_word_index = (100 * srng.uniform((1, n_batch))).astype('int64')
batch_dim = T.arange(n_batch)
wemb = theano.shared(np.arange(100*5).reshape(100,5))

new_end_seq_tensor = T.set_subtensor(a[batch_dim][neg_ending_subs_index], wemb[substi_word_index])

test1 = theano.function([a, neg_ending_subs_index], new_end_seq_tensor)

In [47]:
tensorA = np.random.rand(7, 8, 5)
sub_index_ls = np.asarray([2,6,4,3,2,2,0]).astype('int32')
result = test1(tensorA, sub_index_ls)

ValueError: array is not broadcastable to correct shape
Apply node that caused the error: AdvancedIncSubtensor1{inplace,set}(AdvancedSubtensor1.0, Reshape{3}.0, <TensorType(int32, vector)>)
Toposort index: 12
Inputs types: [TensorType(float64, 3D), TensorType(int64, (True, False, False)), TensorType(int32, vector)]
Inputs shapes: [(7, 8, 5), (1, 7, 5), (7,)]
Inputs strides: [(320, 40, 8), (280, 40, 8), (4,)]
Inputs values: ['not shown', 'not shown', 'not shown']
Outputs clients: [['output']]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/home-nfs/jontsai/anaconda/lib/python2.7/site-packages/IPython/kernel/zmq/kernelbase.py", line 252, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/home-nfs/jontsai/anaconda/lib/python2.7/site-packages/IPython/kernel/zmq/kernelbase.py", line 213, in dispatch_shell
    handler(stream, idents, msg)
  File "/home-nfs/jontsai/anaconda/lib/python2.7/site-packages/IPython/kernel/zmq/kernel

In [43]:
result[2]

array([[ 235.,  236.,  237.,  238.,  239.],
       [ 235.,  236.,  237.,  238.,  239.],
       [ 235.,  236.,  237.,  238.,  239.],
       [ 235.,  236.,  237.,  238.,  239.],
       [ 235.,  236.,  237.,  238.,  239.],
       [ 235.,  236.,  237.,  238.,  239.],
       [ 235.,  236.,  237.,  238.,  239.],
       [ 235.,  236.,  237.,  238.,  239.]])

In [67]:
a = T.tensor3()
b = T.ivector()
c = T.ivector()
d = T.ivector()
e = theano.shared(np.random.rand(100, 5))

f = T.set_subtensor(a[b,c], e[d])
test = theano.function([a,b,c,d], f)


In [72]:
a_ = np.arange(6*15*5).reshape(6,15,5)
b_ = np.arange(6).astype('int32')
c_ = np.random.randint(15, size=(6,)).astype('int32')
d_ = np.random.randint(100, size=(6,)).astype('int32')
result = test(a_, b_, c_, d_)

In [74]:
print c_
print d_

[14  0  0 14 14  5]
[62 69 37 24 60 33]


In [80]:
result[1][0]

array([ 0.3669229 ,  0.0759273 ,  0.34172383,  0.78630821,  0.60700849])

In [3]:
import cPickle as pickle
index2word_dic = pickle.load(open('../../data/pickles/ROC_train_index_dict.pkl'))

wemb_matrix = pickle.load(open('../../data/pickles/index_wemb_matrix.pkl'))

train_set = pickle.load(open('../../data/pickles/train_index_corpus.pkl'))

In [44]:
# Title + story plot 
import numpy as np

story_set = np.asarray(train_set[0])

story_n = len(end_set)
story_rep_matrix = np.zeros((story_n, 300))
for i in range(story_n):
    story_rep_matrix[i] = np.sum(np.sum(wemb_matrix[story_set[i][k][j]] for j in range(len(story_set[i][k]))) for k in range(5))/5.0 

In [45]:
# story plot 

story_set = np.asarray(train_set[0])

story_n = len(end_set)
storyplot_rep_matrix = np.zeros((story_n, 300))
for i in range(story_n):
    storyplot_rep_matrix[i] = np.sum(np.sum(wemb_matrix[story_set[i][k][j]] for j in range(len(story_set[i][k]))) for k in range(1, 5))/4.0 

In [47]:
import numpy as np

end_set = np.asarray(train_set[1])

story_n = len(end_set)
end_rep_matrix = np.zeros((story_n, 300))
for i in range(story_n):
    end_rep_matrix[i] = np.sum(wemb_matrix[end_set[i]], axis = 0)/(len(end_set[i])) 

In [119]:
new_train_set[0]

[array([1, 2]),
 array([3, 4, 5, 6, 1, 7]),
 array([ 3,  8,  1,  9, 10,  7]),
 array([11, 12, 13, 14,  5, 15,  8, 16,  7]),
 array([14,  5, 17, 18, 19, 20, 21, 22, 23,  7]),
 array([24, 25, 26, 18,  3, 27, 22, 28,  7])]

In [122]:
# story plot + ending


story_set = new_train_set

story_n = len(end_set)
storyplotNend_rep_matrix = np.zeros((story_n, 300))
for i in range(story_n):
    storyplotNend_rep_matrix[i] = np.sum(np.sum(wemb_matrix[story_set[i][k]],axis = 0) for k in range(1,6))/5.0 

In [42]:
def cos_simi(rep_matrix, story_n):    
    normalized_matrix_vec = np.linalg.norm(rep_matrix, axis = 1)


    most_similar_pair = np.zeros((story_n, 1))
    for i in range(story_n):
        dot_vec = np.dot(rep_matrix[i], rep_matrix.T)
        norm_vec = np.asarray([normalized_matrix_vec[i] * normalized_matrix_vec[j] for j in range(story_n)])
        cos_vec = dot_vec / norm_vec
        most_similar_pair[i] = np.argsort(cos_vec)[-2]
    return most_similar_pair
        

In [145]:
a = np.array([1,2,34]).reshape(-1,1)
b = np.array([1,2,33]).reshape(-1,1)

c = np.dot(a,b.T)
d = np.argmax(c, axis = 0)
print d.shape

(3,)


In [48]:
most_simi_story_pair = cos_simi(story_rep_matrix, story_n)

In [49]:
most_simi_storyplot_pair = cos_simi(storyplot_rep_matrix, story_n)
most_simi_end_pair = cos_simi(end_rep_matrix, story_n)

In [124]:
most_simi_storyplotNend_pair = cos_simi(storyplotNend_rep_matrix, story_n)

In [68]:
import cPickle as pickle
pickle.dump(most_simi_story_pair.reshape(-1,).astype('int32'), open('../../data/pickles/most_simi_story_pair.pkl', 'w'))
pickle.dump(most_simi_storyplot_pair.reshape(-1,).astype('int32'), open('../../data/pickles/most_simi_storyplot_pair.pkl','w'))
pickle.dump(most_simi_end_pair.reshape(-1,).astype('int32'), open('../../data/pickles/most_simi_end_pair.pkl','w'))

In [116]:
pickle.dump(most_simi_storyplotNend_pair.reshape(-1,).astype('int32'), open('../../data/pickles/most_simi_storyplotNend_pair.pkl', 'w'))

In [71]:
def convert(nump_matrix):
    return nump_matrix.reshape(-1,).astype('int32')

most_simi_end_pair = convert(most_simi_end_pair)
most_simi_story_pair = convert(most_simi_story_pair)
most_simi_storyplot_pair = convert(most_simi_storyplot_pair)


In [127]:
most_simi_storyPNend_pair = convert(most_simi_storyplotNend_pair)

In [128]:
most_simi_storyPNend_pair[:10]

array([ 3665,   899, 11902, 10411, 18049, 32872, 23746,  4604, 15949, 34794], dtype=int32)

In [172]:
rand_start = np.random.randint(story_n)
for i in range(rand_start, rand_start + 30):
    print "instance #",i - rand_start
    print " | ".join([" ".join([index2word_dic[new_train_set[i][j][k]] for k in range(len(new_train_set[i][j]))]) for j in range(6)])
    print "==================most similar end pairs===================="
    print " | ".join([" ".join([index2word_dic[new_train_set[most_simi_storyPNend_pair[i]][j][k]] for k in range(len(new_train_set[most_simi_storyPNend_pair[i]][j]))]) for j in range(6)])
    print ""


instance # 0
dental adventures | one day , sally was eating nuts and heard a crack . | she had bitten into a shell and cracked her tooth ! | the tooth hurt very badly . | she called her dentist . | the dentist saw her later that day and fixed her tooth .
==================most similar end pairs====================
loose tooth | tricia had a loose tooth . | she was eating her cereal one morning . | her tooth was hanging on by just a little bit . | tricia bit into something hard . | she realized her tooth had come out in her cereal .

instance # 1
meeting an old friend | i recently met an old friend . | i accompanied him while he ran various errands . | we went to his job , his house , and a thrift shop . | i quite enjoyed playing around in the thrift shop . | the day wasn 't exciting , but i enjoyed my friend 's company .
==================most similar end pairs====================
keeping lips closed | i was walking to the local coffee shop . | as soon as i looked around , i can into m

In [130]:
most_simi_end = pickle.load(open('../../data/pickles/most_simi_end_pair.pkl'))

(45502,)


In [131]:
type(most_simi_end[0])

numpy.int32

In [18]:
same_pair_dict = {}

for i in range(len(compare_vec)):
    if compare_vec[i] == 0:
        same_pair_dict[i] = most_simi_story_pair[i]
        

In [7]:
import theano
import theano.tensor as T

a = T.matrix()
a_norm = T.sqrt(T.sum(T.sqr(a),axis = 1,keepdims = True))
b = a/a_norm

test = theano.function([a],b)

In [8]:
import numpy as np
a_t = np.arange(-1,5).reshape(3,2)

theano_version = test(a_t)
numpy_version = a_t/np.sum(a_t, axis = 1).reshape(-1,1)
np_lin_version = a_t/np.linalg.norm(a_t, axis = 1).reshape(-1,1)
print a_t
print theano_version
print numpy_version
print np_lin_version

[[-1  0]
 [ 1  2]
 [ 3  4]]
[[-1.          0.        ]
 [ 0.4472136   0.89442719]
 [ 0.6         0.8       ]]
[[1 0]
 [0 0]
 [0 0]]
[[-1.          0.        ]
 [ 0.4472136   0.89442719]
 [ 0.6         0.8       ]]


In [25]:
most_simi_story_pair[:10]

array([ 20210.,  11891.,  16703.,  27791.,  40423.,  25657.,  11257.,
        40423.,  40023.,   6907.])

In [26]:
most_simi_story_pair[42605]

19170.0

In [ ]:
import numpy as np

whole_set = train_set[1]

story_n = len(end_set)
ending_rep_matrix = np.zeros((story_n, 300))
for i in range(story_n):
    ending_rep_matrix[i] = np.sum([wemb_matrix[end_set[i][j]] for j in range(len(end_set[i]))], axis = 0)/float(len(end_set[i]))


In [159]:
train_set = pickle.load(open('../../data/pickles/train_index_corpus.pkl'))
train_story = train_set[0]
train_ending = train_set[1]
n_train = len(train_ending)
stories_indices = np.random.randint(n_train, size=(5,))
adv_end_rep_batch = np.random.rand(5, 300)
for i in range(len(train_ending)):
    end_rep_matrix[i] = np.sum(wemb_matrix[train_ending[i]], axis = 0) / (len(train_ending[i]))




# '''part III print out story and the most similar end correspondingly'''
# for i in range(5):
#     index = stories_indices[i]
#     story_string = " | ".join([" ".join(index2word_dict[train_story[index][j][k]] for k in range(len(train_story[index][j])))] for j in range(5))
#     story_end = " ".join(index2word_dict[train_ending[index][k]] for k in range(len(train_ending[index])))
#     generated_end = " ".join(index2word_dict[train_ending[index_list[i]][k]] for k in range(len(train_ending[index_list[i]])))

#     print story_string + "#END#" + story_end
#     print "adv model generated:" + generated_end

In [160]:
norm_end_rep_matrix = np.linalg.norm(end_rep_matrix, axis = 1).reshape(-1,1)
norm_adv_end_rep = np.linalg.norm(adv_end_rep_batch, axis = 1).reshape(-1,1)
# norm_denominator_matrix.shape = (45503, 5)
norm_denominator_matrix = np.dot(norm_end_rep_matrix, norm_adv_end_rep.T)

In [161]:
# dot_prod.shape = (45503, 5)
dot_prod = np.dot(end_rep_matrix, adv_end_rep_batch.T)

# cos_simi_matrix.shape = (45503, 5)
cos_simi_matrix = dot_prod / norm_denominator_matrix


In [175]:
index_list = np.argmax(cos_simi_matrix, axis = 0)
index2word_dict = index2word_dic

In [2]:
SGD_LR_ls = [0.001, 0.01, 0.005, 0.002]

Adam_LR_ls = [0.001, 0.002, 0.0005, 0.0001]

for i in range(len(SGD_LR_ls)):
    for j in range(len(Adam_LR_ls)):
        f = open('./AdvInvLogSepSGDAdamLR'+str(i)+str(j)+'.job','w')
        
        f.write('#!/bin/bash\n')
        f.write('#$ -S /bin/bash\n')
        f.write('#$ -M jontsai@uchicago.edu\n')
        f.write('#$ -N AdvInvLogSepSGDAdamLR'+str(i)+str(j)+'\n')
        f.write('#$ -m beasn\n')
        f.write('#$ -o AdvInvLogSepSGDAdamLR'+str(i)+str(j)+'.out\n')
        f.write('#$ -e AdvInvLogSepSGDAdamLR'+str(i)+str(j)+'.err\n')
        f.write('#$ -r n\n')
        f.write('#$ -cwd\n')
        f.write('SETTING1="300"\n')
        f.write('SETTING2="0.8"\n')
        f.write('SETTING3="150"\n')
        f.write('SETTING4="100"\n')
        f.write('SETTING5="512x512"\n')
        f.write('SETTING6="'+str(SGD_LR_ls[i])+'"\n')
        f.write('SETTING7="'+str(Adam_LR_ls[j])+'"\n')
        f.write('SETTING8="sgd"\n')
        f.write('SETTING9="adam"\n')
        f.write('SETTING10="default"\n')
        f.write('SETTING11="0"\n')
        f.write('SETTING12="0.005"\n')
        f.write('export OMP_NUM_THREADS=2\n')
        f.write('export OPENBLAS_NUM_THREADS=2\n')
        f.write('echo "Start - `date`"\n')
        f.write('/home-nfs/jontsai/anaconda/bin/python BLSTM_inverseAtt_onlyAdvOnVal.py \\\n')
        f.write('$SETTING1 $SETTING2 $SETTING3 $SETTING4 $SETTING5 $SETTING6 $SETTING7 $SETTING8 $SETTING9 $SETTING10 $SETTING11 $SETTING12\n')
        f.write('echo "End - `date`"\n')
        f.close()


In [12]:
SGD_LR_ls = [0.001, 0.002, 0.003, 0.004]

Adam_LR_ls = [0.001, 0.002, 0.0005, 0.0001]

for i in range(len(SGD_LR_ls)):
    for j in range(len(Adam_LR_ls)):
        f = open('./AdvInvLogSepAdamAdamLR'+str(i)+str(j)+'.job','w')

        f.write('#!/bin/bash\n')
        f.write('#$ -S /bin/bash\n')
        f.write('#$ -M jontsai@uchicago.edu\n')
        f.write('#$ -N AdvInvLogSepAdamAdamLR'+str(i)+str(j)+'\n')
        f.write('#$ -m beasn\n')
        f.write('#$ -o AdvInvLogSepAdamAdamLR'+str(i)+str(j)+'.out\n')
        f.write('#$ -e AdvInvLogSepAdamAdamLR'+str(i)+str(j)+'.err\n')
        f.write('#$ -r n\n')
        f.write('#$ -cwd\n')
        f.write('SETTING1="300"\n')
        f.write('SETTING2="0.8"\n')
        f.write('SETTING3="150"\n')
        f.write('SETTING4="100"\n')
        f.write('SETTING5="512x512"\n')
        f.write('SETTING6="'+str(SGD_LR_ls[i])+'"\n')
        f.write('SETTING7="'+str(Adam_LR_ls[j])+'"\n')
        f.write('SETTING8="adam"\n')
        f.write('SETTING9="adam"\n')
        f.write('SETTING10="None"\n')
        f.write('SETTING11="0"\n')
        f.write('SETTING12="'+str(Adam_LR_ls[j])+'"\n')
        f.write('SETTING13="euclidean_distance"\n')
        f.write('export OMP_NUM_THREADS=2\n')
        f.write('export OPENBLAS_NUM_THREADS=2\n')
        f.write('echo "Start - `date`"\n')
        f.write('/home-nfs/jontsai/anaconda/bin/python BLSTM_inverseAtt_onlyAdvOnVal.py \\\n')
        f.write('$SETTING1 $SETTING2 $SETTING3 $SETTING4 $SETTING5 $SETTING6 $SETTING7 $SETTING8 $SETTING9 $SETTING10 $SETTING11 $SETTING12 $SETTING13\n')
        f.write('echo "End - `date`"\n')
        f.close()


In [4]:
batchsize_ls = [100, 150, 200, 250]
Adam_LR_ls = [0.001, 0.002, 0.0005, 0.0002]

for i in range(len(batchsize_ls)):
    for j in range(len(Adam_LR_ls)):
        f = open('../hierachical_rnn/HGRU_innermax' +str(i)+str(j)+'.job','w')

        f.write('#!/bin/bash\n')
        f.write('#$ -S /bin/bash\n')
        f.write('#$ -M jontsai@uchicago.edu\n')
        f.write('#$ -N HGRU_innermax'+str(i)+str(j)+'\n')
        f.write('#$ -m beasn\n')
        f.write('#$ -o HGRU_innermax'+str(i)+str(j)+'.out\n')
        f.write('#$ -e HGRU_innermax'+str(i)+str(j)+'.err\n')
        f.write('#$ -r n\n')
        f.write('#$ -cwd\n')
        f.write('SETTING1="300"\n')
        f.write('SETTING2="0.0"\n')
        f.write('SETTING3="'+str(batchsize_ls[i])+'"\n')
        f.write('SETTING4="default"\n')
        f.write('SETTING5="0"\n')
        f.write('SETTING6="'+str(Adam_LR_ls[j])+'"\n')
        f.write('SETTING7="0.5"\n')
        f.write('export OMP_NUM_THREADS=2\n')
        f.write('export OPENBLAS_NUM_THREADS=2\n')
        f.write('echo "Start - `date`"\n')
        f.write('/home-nfs/jontsai/anaconda/bin/python BGRU2BGRU_hingeNearRight.py \\\n')
        f.write('$SETTING1 $SETTING2 $SETTING3 $SETTING4 $SETTING5 $SETTING6 $SETTING7\n')
        f.write('echo "End - `date`"\n')
        f.close()


In [1]:
# HierBLSTM job scripts 

batch_size = [200, 250, 300, 350]
Adam_LR_ls = [0.001, 0.002, 0.0005, 0.0001]

for i in range(len(batch_size)):
    for j in range(len(Adam_LR_ls)):
        f = open('../hierachical_rnn/HierBLSTM_cos'+str(i)+str(j)+'.job','w')

        f.write('#!/bin/bash\n')
        f.write('#$ -S /bin/bash\n')
        f.write('#$ -M jontsai@uchicago.edu\n')
        f.write('#$ -N HierBLSTM_cos'+str(i)+str(j)+'\n')
        f.write('#$ -m beasn\n')
        f.write('#$ -o HierBLSTM_cos'+str(i)+str(j)+'.out\n')
        f.write('#$ -e HierBLSTM_cos'+str(i)+str(j)+'.err\n')
        f.write('#$ -r n\n')
        f.write('#$ -cwd\n')
        f.write('SETTING1="300"\n')
        f.write('SETTING2="'+str(batch_size[i])+'"\n')
        f.write('SETTING3="default"\n')
        f.write('SETTING4="0"\n')
        f.write('SETTING5="'+str(Adam_LR_ls[j])+'"\n')
        f.write('SETTING6="2.0"\n')
        f.write('export OMP_NUM_THREADS=2\n')
        f.write('export OPENBLAS_NUM_THREADS=2\n')
        f.write('echo "Start - `date`"\n')
        f.write('/home-nfs/jontsai/anaconda/bin/python BLSTM_hingeCosine.py \\\n')
        f.write('$SETTING1 $SETTING2 $SETTING3 $SETTING4 $SETTING5 $SETTING6\n')
        f.write('echo "End - `date`"\n')
        f.close()

In [3]:
# HierBLSTM job scripts 

batch_size = [200, 250, 300, 350]
Adam_LR_ls = [0.002, 0.001, 0.0005, 0.0001]

for i in range(len(batch_size)):
    for j in range(len(Adam_LR_ls)):
        f = open('../hierachical_rnn/HierBGRU_cos'+str(i)+str(j)+'.job','w')

        f.write('#!/bin/bash\n')
        f.write('#$ -S /bin/bash\n')
        f.write('#$ -M jontsai@uchicago.edu\n')
        f.write('#$ -N HierBGRU_cos'+str(i)+str(j)+'\n')
        f.write('#$ -m beasn\n')
        f.write('#$ -o HierBGRU_cos'+str(i)+str(j)+'.out\n')
        f.write('#$ -e HierBGRU_cos'+str(i)+str(j)+'.err\n')
        f.write('#$ -r n\n')
        f.write('#$ -cwd\n')
        f.write('SETTING1="300"\n')
        f.write('SETTING2="0.0"\n')
        f.write('SETTING3="'+str(batch_size[i])+'"\n')
        f.write('SETTING4="default"\n')
        f.write('SETTING5="0"\n')
        f.write('SETTING6="'+str(Adam_LR_ls[j])+'"\n')
        f.write('SETTING7="2.0"\n')
        f.write('export OMP_NUM_THREADS=2\n')
        f.write('export OPENBLAS_NUM_THREADS=2\n')
        f.write('echo "Start - `date`"\n')
        f.write('/home-nfs/jontsai/anaconda/bin/python BGRU2BGRU_hingeCosine.py \\\n')
        f.write('$SETTING1 $SETTING2 $SETTING3 $SETTING4 $SETTING5 $SETTING6 $SETTING7\n')
        f.write('echo "End - `date`"\n')
        f.close()

In [4]:
import numpy as np

best_val_acc_ls = np.zeros((4,4))

best_test_acc_ls = np.zeros((4,4))

for i in range(4):
    for j in range(4):
        f = open('../hierachical_rnn/HierBGRU_cos'+str(i)+str(j)+'.out','r')
        best_val_acc = 0.0
        best_test_acc = 0.0
        for line in f:
            if line.startswith('accuracy is'):
                val_acc = float(line.split(' ')[-2])
                if val_acc >= best_val_acc:
                    best_val_acc = val_acc
            if line.startswith('test set accuracy'):
                test_acc = float(line.split(' ')[-2])
                if test_acc >= best_test_acc:
                    best_test_acc = test_acc
        best_val_acc_ls[i][j] = best_val_acc
        best_test_acc_ls[i][j] = best_test_acc
        f.close()

In [5]:
print best_val_acc_ls
print best_test_acc_ls


[[ 55.47835382  56.38695885  55.2645644   56.49385355]
 [ 54.14216996  56.38695885  57.24211652  55.31801176]
 [ 57.88348477  56.28006414  56.4404062   56.33351149]
 [ 54.99732763  56.65419562  56.76109033  56.11972207]]
[[ 54.94388028  55.1576697   53.23356494  54.83698557]
 [ 52.37840727  54.03527525  54.89043292  55.10422234]
 [ 55.1576697   54.62319615  54.78353821  54.5697488 ]
 [ 52.96632817  55.47835382  53.66114377  55.1576697 ]]


In [8]:
for i in range(4):
    for j in range(4):
        f = open('../hierachical_rnn/HierBLSTM_cos'+str(i)+str(j)+'.out','r')
        epoch = 0
        for line in f:
            if line.startswith('epoch  '):
                tokens = line.split()
                epoch_num = int(tokens[-2])
                if epoch_num > epoch:
                    epoch = epoch_num
        print epoch
        f.close()

27
27
27
27
27
26
32
33
32
32
32
32
26
26
26
26


In [2]:
#HierBGRU_hingeDNN large batch size experiments job scripts
batch_size = [200, 250, 300, 350]
Adam_LR_ls = [0.002, 0.001, 0.0005, 0.00025]

for i in range(len(batch_size)):
    for j in range(len(Adam_LR_ls)):
        f = open('../hierachical_rnn/HierBGRU_hingeDNN'+str(i)+str(j)+'.job','w')
        f.write('#!/bin/bash\n')
        f.write('#$ -S /bin/bash\n')
        f.write('#$ -M jontsai@uchicago.edu\n')
        f.write('#$ -N HierBGRU_DNN'+str(i)+str(j)+'\n')
        f.write('#$ -m beasn\n')
        f.write('#$ -o HierBGRU_DNN'+str(i)+str(j)+'.out\n')
        f.write('#$ -e HierBGRU_DNN'+str(i)+str(j)+'.err\n')
        f.write('#$ -r n\n')
        f.write('#$ -cwd\n')
        f.write('SETTING1="300"\n')
        f.write('SETTING2="0.05"\n')
        f.write('SETTING3="'+str(batch_size[i])+'"\n')
        f.write('SETTING4="None"\n')
        f.write('SETTING5="0"\n')
        f.write('SETTING6="'+str(Adam_LR_ls[j])+'"\n')
        f.write('SETTING7="1.0"\n')
        f.write('SETTING8="last"\n')
        f.write('export OMP_NUM_THREADS=2\n')
        f.write('export OPENBLAS_NUM_THREADS=2\n')
        f.write('echo "Start - `date`"\n')
        f.write('/home-nfs/jontsai/anaconda/bin/python HierBGRU_hingeDNN.py \\\n')
        f.write('$SETTING1 $SETTING2 $SETTING3 $SETTING4 $SETTING5 $SETTING6 $SETTING7 $SETTING8\n')
        f.write('echo "End - `date`"\n')
        f.close()

In [28]:
a = T.vector()
b = T.vector()
c = a/b

test = theano.function([a,b], c)


In [30]:
a = np.arange(5)
b = np.arange(5)
c = test(a,b)
print c.shape
print c

(5,)
[ nan   1.   1.   1.   1.]


In [ ]:
'''
Recurrent network example.  Trains a 2 layered LSTM network to learn
text from a user-provided input file. The network can then be used to generate
text using a short string as seed (refer to the variable generation_phrase).
This example is partly based on Andrej Karpathy's blog
(http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
and a similar example in the Keras package (keras.io).
The inputs to the network are batches of sequences of characters and the corresponding
targets are the characters in the text shifted to the right by one. 
Assuming a sequence length of 5, a training point for a text file
"The quick brown fox jumps over the lazy dog" would be
INPUT : 'T','h','e',' ','q'
OUTPUT: 'u'
The loss function compares (via categorical crossentropy) the prediction
with the output/target.
Also included is a function to generate text using the RNN given the first 
character.  
About 20 or so epochs are necessary to generate text that "makes sense".
Written by @keskarnitish
Pre-processing of text uses snippets of Karpathy's code (BSD License)
'''

from __future__ import print_function


import numpy as np
import theano
import theano.tensor as T
import lasagne
import urllib2 #For downloading the sample text file. You won't need this if you are providing your own file.
try:
    in_text = urllib2.urlopen('https://s3.amazonaws.com/text-datasets/nietzsche.txt').read()
    #You can also use your own file
    #The file must be a simple text file.
    #Simply edit the file name below and uncomment the line.  
    #in_text = open('your_file.txt', 'r').read()
    in_text = in_text.decode("utf-8-sig").encode("utf-8")
except Exception as e:
    print("Please verify the location of the input file/URL.")
    print("A sample txt file can be downloaded from https://s3.amazonaws.com/text-datasets/nietzsche.txt")
    raise IOError('Unable to Read Text')

generation_phrase = "The quick brown fox jumps" #This phrase will be used as seed to generate text.

#This snippet loads the text file and creates dictionaries to 
#encode characters into a vector-space representation and vice-versa. 
chars = list(set(in_text))
data_size, vocab_size = len(in_text), len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

#Lasagne Seed for Reproducibility
lasagne.random.set_rng(np.random.RandomState(1))

# Sequence Length
SEQ_LENGTH = 20

# Number of units in the two hidden (LSTM) layers
N_HIDDEN = 512

# Optimization learning rate
LEARNING_RATE = .01

# All gradients above this will be clipped
GRAD_CLIP = 100

# How often should we check the output?
PRINT_FREQ = 1000

# Number of epochs to train the net
NUM_EPOCHS = 50

# Batch Size
BATCH_SIZE = 128


def gen_data(p, batch_size = BATCH_SIZE, data=in_text, return_target=True):
    '''
    This function produces a semi-redundant batch of training samples from the location 'p' in the provided string (data).
    For instance, assuming SEQ_LENGTH = 5 and p=0, the function would create batches of 
    5 characters of the string (starting from the 0th character and stepping by 1 for each semi-redundant batch)
    as the input and the next character as the target.
    To make this clear, let us look at a concrete example. Assume that SEQ_LENGTH = 5, p = 0 and BATCH_SIZE = 2
    If the input string was "The quick brown fox jumps over the lazy dog.",
    For the first data point,
    x (the inputs to the neural network) would correspond to the encoding of 'T','h','e',' ','q'
    y (the targets of the neural network) would be the encoding of 'u'
    For the second point,
    x (the inputs to the neural network) would correspond to the encoding of 'h','e',' ','q', 'u'
    y (the targets of the neural network) would be the encoding of 'i'
    The data points are then stacked (into a three-dimensional tensor of size (batch_size,SEQ_LENGTH,vocab_size))
    and returned. 
    Notice that there is overlap of characters between the batches (hence the name, semi-redundant batch).
    '''
    x = np.zeros((batch_size,SEQ_LENGTH,vocab_size))
    y = np.zeros(batch_size)

    for n in range(batch_size):
        ptr = n
        for i in range(SEQ_LENGTH):
            x[n,i,char_to_ix[data[p+ptr+i]]] = 1.
        if(return_target):
            y[n] = char_to_ix[data[p+ptr+SEQ_LENGTH]]
    return x, np.array(y,dtype='int32')



def main(num_epochs=NUM_EPOCHS):
    print("Building network ...")
   
    # First, we build the network, starting with an input layer
    # Recurrent layers expect input of shape
    # (batch size, SEQ_LENGTH, num_features)

    l_in = lasagne.layers.InputLayer(shape=(None, None, vocab_size))

    # We now build the LSTM layer which takes l_in as the input layer
    # We clip the gradients at GRAD_CLIP to prevent the problem of exploding gradients. 

    l_forward_1 = lasagne.layers.LSTMLayer(
        l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
        nonlinearity=lasagne.nonlinearities.tanh)

    l_forward_2 = lasagne.layers.LSTMLayer(
        l_forward_1, N_HIDDEN, grad_clipping=GRAD_CLIP,
        nonlinearity=lasagne.nonlinearities.tanh,
        only_return_final=True)

    # The output of l_forward_2 of shape (batch_size, N_HIDDEN) is then passed through the softmax nonlinearity to 
    # create probability distribution of the prediction
    # The output of this stage is (batch_size, vocab_size)
    l_out = lasagne.layers.DenseLayer(l_forward_2, num_units=vocab_size, W = lasagne.init.Normal(), nonlinearity=lasagne.nonlinearities.softmax)

    # Theano tensor for the targets
    target_values = T.ivector('target_output')
    
    # lasagne.layers.get_output produces a variable for the output of the net
    network_output = lasagne.layers.get_output(l_out)

    # The loss function is calculated as the mean of the (categorical) cross-entropy between the prediction and target.
    cost = T.nnet.categorical_crossentropy(network_output,target_values).mean()

    # Retrieve all parameters from the network
    all_params = lasagne.layers.get_all_params(l_out,trainable=True)

    # Compute AdaGrad updates for training
    print("Computing updates ...")
    updates = lasagne.updates.adagrad(cost, all_params, LEARNING_RATE)

    # Theano functions for training and computing cost
    print("Compiling functions ...")
    train = theano.function([l_in.input_var, target_values], cost, updates=updates, allow_input_downcast=True)
    compute_cost = theano.function([l_in.input_var, target_values], cost, allow_input_downcast=True)

    # In order to generate text from the network, we need the probability distribution of the next character given
    # the state of the network and the input (a seed).
    # In order to produce the probability distribution of the prediction, we compile a function called probs. 
    
    probs = theano.function([l_in.input_var],network_output,allow_input_downcast=True)

    # The next function generates text given a phrase of length at least SEQ_LENGTH.
    # The phrase is set using the variable generation_phrase.
    # The optional input "N" is used to set the number of characters of text to predict. 

    def try_it_out(N=200):
        '''
        This function uses the user-provided string "generation_phrase" and current state of the RNN generate text.
        The function works in three steps:
        1. It converts the string set in "generation_phrase" (which must be over SEQ_LENGTH characters long) 
           to encoded format. We use the gen_data function for this. By providing the string and asking for a single batch,
           we are converting the first SEQ_LENGTH characters into encoded form. 
        2. We then use the LSTM to predict the next character and store it in a (dynamic) list sample_ix. This is done by using the 'probs'
           function which was compiled above. Simply put, given the output, we compute the probabilities of the target and pick the one 
           with the highest predicted probability. 
        3. Once this character has been predicted, we construct a new sequence using all but first characters of the 
           provided string and the predicted character. This sequence is then used to generate yet another character.
           This process continues for "N" characters. 
        To make this clear, let us again look at a concrete example. 
        Assume that SEQ_LENGTH = 5 and generation_phrase = "The quick brown fox jumps". 
        We initially encode the first 5 characters ('T','h','e',' ','q'). The next character is then predicted (as explained in step 2). 
        Assume that this character was 'J'. We then construct a new sequence using the last 4 (=SEQ_LENGTH-1) characters of the previous
        sequence ('h','e',' ','q') , and the predicted letter 'J'. This new sequence is then used to compute the next character and 
        the process continues.
        '''

        assert(len(generation_phrase)>=SEQ_LENGTH)
        sample_ix = []
        x,_ = gen_data(len(generation_phrase)-SEQ_LENGTH, 1, generation_phrase,0)

        for i in range(N):
            # Pick the character that got assigned the highest probability
            ix = np.argmax(probs(x).ravel())
            # Alternatively, to sample from the distribution instead:
            # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
            sample_ix.append(ix)
            x[:,0:SEQ_LENGTH-1,:] = x[:,1:,:]
            x[:,SEQ_LENGTH-1,:] = 0
            x[0,SEQ_LENGTH-1,sample_ix[-1]] = 1. 

        random_snippet = generation_phrase + ''.join(ix_to_char[ix] for ix in sample_ix)    
        print("----\n %s \n----" % random_snippet)


    
    print("Training ...")
    print("Seed used for text generation is: " + generation_phrase)
    p = 0
    try:
        for it in xrange(data_size * num_epochs / BATCH_SIZE):
            try_it_out() # Generate text using the p^th character as the start. 
            
            avg_cost = 0;
            for _ in range(PRINT_FREQ):
                x,y = gen_data(p)
                
                #print(p)
                p += SEQ_LENGTH + BATCH_SIZE - 1 
                if(p+BATCH_SIZE+SEQ_LENGTH >= data_size):
                    print('Carriage Return')
                    p = 0;
                

                avg_cost += train(x, y)
            print("Epoch {} average loss = {}".format(it*1.0*PRINT_FREQ/data_size*BATCH_SIZE, avg_cost / PRINT_FREQ))
                    
    except KeyboardInterrupt:
        pass

if __name__ == '__main__':
    main()

In [1]:
import urllib2 #For downloading the sample text file. You won't need this if you are providing your own file.

try:
    in_text = urllib2.urlopen('https://s3.amazonaws.com/text-datasets/nietzsche.txt').read()
    #You can also use your own file
    #The file must be a simple text file.
    #Simply edit the file name below and uncomment the line.  
    #in_text = open('your_file.txt', 'r').read()
    in_text = in_text.decode("utf-8-sig").encode("utf-8")
except Exception as e:
    print("Please verify the location of the input file/URL.")
    print("A sample txt file can be downloaded from https://s3.amazonaws.com/text-datasets/nietzsche.txt")
    raise IOError('Unable to Read Text')

generation_phrase = "The quick brown fox jumps" #This phrase will be used as seed to generate text.

#This snippet loads the text file and creates dictionaries to 
#encode characters into a vector-space representation and vice-versa. 
chars = list(set(in_text))
data_size, vocab_size = len(in_text), len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }


In [4]:
print len(in_text)
print in_text[:100]

600901
PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all ph


In [4]:
import theano
import theano.tensor as T
import BLSTM_sequence
import numpy as np
import lasagne
a = T.matrix(dtype='int64')
a_reshape = a.dimshuffle(0,1,'x')
b = T.matrix(dtype=theano.config.floatX)

wemb = theano.shared(np.random.rand(20,2))
encoder = BLSTM_sequence.BlstmEncoder(LSTMLAYER_1_UNITS = 2, wemb_trainable = False)
encoder.build_model(wemb)

seq = lasagne.layers.get_output(encoder.output, {encoder.l_in: a_reshape,
                                                 encoder.l_mask: b})

test = theano.function([a,b], seq)

In [5]:
len_ls = np.random.randint(6, size = (5,))
batch_ls = []
for i in range(5):
    batch_ls.append(list(np.random.randint(20, size=(len_ls[i],))))


In [7]:
import utils

batch_matrix = utils.padding(batch_ls)
batch_mask = utils.mask_generator(batch_ls)
print batch_matrix
print batch_mask

[[15  7 17 16  3]
 [ 7  0  0  0  0]
 [ 7 15  0  0  0]
 [14 19  0  0  0]
 [ 4 15 17  8  0]]
[[ 1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.]]


In [8]:
test(batch_matrix, batch_mask)

array([[[ 0.13653159,  0.70147757],
        [ 0.00950921,  0.58593329],
        [ 0.0442238 ,  0.72475287],
        [-0.03927389,  0.78916293],
        [-0.18472336,  0.84670197]],

       [[-0.0461301 ,  0.20130033],
        [-0.03104189,  0.09977244],
        [-0.03104189,  0.09977244],
        [-0.03104189,  0.09977244],
        [-0.03104189,  0.09977244]],

       [[ 0.02154436,  0.31594813],
        [ 0.03461041,  0.60768658],
        [-0.02225799,  0.37655755],
        [-0.02225799,  0.37655755],
        [-0.02225799,  0.37655755]],

       [[ 0.13216289,  0.62139024],
        [-0.0686795 ,  0.83251609],
        [-0.05181519,  0.54631201],
        [-0.05181519,  0.54631201],
        [-0.05181519,  0.54631201]],

       [[-0.01564802,  0.31434353],
        [ 0.0630465 ,  0.68519586],
        [-0.02255314,  0.57926313],
        [-0.1359491 ,  0.3418934 ],
        [-0.1499376 ,  0.31680127]]])

In [17]:
a = T.matrix()


a_b = a.dimshuffle(0,'x',1)*T.ones((a.shape[0],2,a.shape[1]))



test = theano.function([a], a_b)




In [18]:
a_t = np.arange(6).reshape(3,2)

test(a_t)

array([[[ 0.,  1.],
        [ 0.,  1.]],

       [[ 2.,  3.],
        [ 2.,  3.]],

       [[ 4.,  5.],
        [ 4.,  5.]]])

In [26]:
l_pair_in = lasagne.layers.InputLayer(shape = (None, 10))

# we cache the encoded result to pick the max score negative sample from them later
l_hid1 = lasagne.layers.DenseLayer(l_pair_in, num_units = 1024, nonlinearity = lasagne.nonlinearities.tanh)

l_hid = lasagne.layers.DenseLayer(l_hid1, num_units=1, nonlinearity = lasagne.nonlinearities.tanh)

matrix_input = T.tensor3()

reshaped_input = matrix_input.reshape((matrix_input.shape[0]*matrix_input.shape[1], -1))
score = lasagne.layers.get_output(l_hid, {l_pair_in:reshaped_input})

test = theano.function([matrix_input], score)

In [34]:
import theano.tensor.nlinalg as TL
a = T.vector()
dim = a.shape[0]
b = T.eye(dim)
c = a*b


test = theano.function([a], c)

In [10]:
import theano
import theano.tensor as T

import numpy as np

a = T.tensor3()
b = T.tensor3()


c = T.batched_dot(a,b)

test = theano.function([a,b], c)

In [12]:
a_t = np.arange(3*4).reshape(3,4,1)
b_t = np.arange(3*4).reshape(3,1,4)

c_t = test(a_t, b_t)

print c_t.shape

(3, 4, 4)
